# Preparation of GWR - Data

## Libraries and Settings

In [1]:
# Libraries
import os
import json
import folium
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import date

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

print(os.getcwd())

/workspaces/spatial_data_analysis/05_Python_GWR_Data


## Download GWR data

Source: https://public.madd.bfs.admin.ch

In [11]:
# Download latest file for the Canton of Zurich
url = "https://public.madd.bfs.admin.ch/buildings_zh.geojson" #when i want more data i just need to change the url
response = requests.get(url)

# Ensure response is valid
if response.status_code == 200:
    # Open file in write mode and write the response content
    with open('buildings_zh.geojson', 'wb') as file:
        file.write(response.content)
else:
    print(f"Failed to download file, status code: {response.status_code}")

## Import latest GWR Data

In [12]:
# Load json file
with open('buildings_zh.geojson') as f:
    data = json.load(f)

# Flatten nested json data
df_orig = pd.json_normalize(data, record_path=['features'])
df_orig

# Remove prefix
df_orig.columns = df_orig.columns.str.replace('properties.', '')
df_orig.columns = df_orig.columns.str.replace('geometry.', '')

# Create copy
df = df_orig.copy()
df.head()

,type,egid,buildingStatus,buildingCategory,buildingClass,municipalityNumber,municipalityName,canton,type,coordinates
0,Feature,1,1004,1040,1271,2,Affoltern am Albis,ZH,Point,"[2676523, 1235843]"
1,Feature,10,1004,1020,1110,2,Affoltern am Albis,ZH,Point,"[2676440.038, 1235973.538]"
2,Feature,100,1004,1020,1110,2,Affoltern am Albis,ZH,Point,"[2676765.706, 1236101.658]"
3,Feature,1000,1007,1030,1110,2,Affoltern am Albis,ZH,Point,"[2676644.881, 1236812.863]"
4,Feature,10000,1004,1030,1121,291,Andelfingen,ZH,Point,"[2693391.65, 1272255.267]"


## Separate Swiss LV95 coordinates

In [13]:
# Separate coordinates
df['x_coords'] = pd.DataFrame(df['coordinates'].tolist(), columns=['x_coord', 'y_coord'])['x_coord']
df['y_coords'] = pd.DataFrame(df['coordinates'].tolist(), columns=['x_coord', 'y_coord'])['y_coord']

# Remove column 'coordinates'
df = df.drop(['coordinates'], axis=1)
df

,type,egid,buildingStatus,buildingCategory,buildingClass,municipalityNumber,municipalityName,canton,type,x_coords,y_coords
0,Feature,1,1004,1040,1271,2,Affoltern am Albis,ZH,Point,2676523.000,1235843.000
1,Feature,10,1004,1020,1110,2,Affoltern am Albis,ZH,Point,2676440.038,1235973.538
2,Feature,100,1004,1020,1110,2,Affoltern am Albis,ZH,Point,2676765.706,1236101.658
3,Feature,1000,1007,1030,1110,2,Affoltern am Albis,ZH,Point,2676644.881,1236812.863
4,Feature,10000,1004,1030,1121,291,Andelfingen,ZH,Point,2693391.650,1272255.267
...,...,...,...,...,...,...,...,...,...,...,...
395118,Feature,99995,1004,1040,0,198,Uster,ZH,Point,2696792.976,1245085.637
395119,Feature,99996,1004,1040,0,198,Uster,ZH,Point,2696718.474,1245039.536
395120,Feature,99997,1004,1040,0,198,Uster,ZH,Point,2696722.040,1245043.472
395121,Feature,99998,1004,1040,0,198,Uster,ZH,Point,2696727.666,1245050.586


## Create WGS84 coordinates

In [14]:
# Create geodataframe and calculate latitude and longitude
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['x_coords'], df['y_coords']), crs="EPSG:2056")

# Convert the Swiss LV95 coordinates to lat & lon
gdf = gdf.to_crs(epsg=4326)

# Get Latitude and Longitude
df['latitude'] = gdf['geometry'].y
df['longitude'] = gdf['geometry'].x

# Show data
df


,type,egid,buildingStatus,buildingCategory,buildingClass,municipalityNumber,municipalityName,canton,type,x_coords,y_coords,latitude,longitude
0,Feature,1,1004,1040,1271,2,Affoltern am Albis,ZH,Point,2676523.000,1235843.000,47.269056,8.449859
1,Feature,10,1004,1020,1110,2,Affoltern am Albis,ZH,Point,2676440.038,1235973.538,47.270239,8.448785
2,Feature,100,1004,1020,1110,2,Affoltern am Albis,ZH,Point,2676765.706,1236101.658,47.271354,8.453110
3,Feature,1000,1007,1030,1110,2,Affoltern am Albis,ZH,Point,2676644.881,1236812.863,47.277764,8.451635
4,Feature,10000,1004,1030,1121,291,Andelfingen,ZH,Point,2693391.650,1272255.267,47.594339,8.680355
...,...,...,...,...,...,...,...,...,...,...,...,...,...
395118,Feature,99995,1004,1040,0,198,Uster,ZH,Point,2696792.976,1245085.637,47.349516,8.719673
395119,Feature,99996,1004,1040,0,198,Uster,ZH,Point,2696718.474,1245039.536,47.349113,8.718677
395120,Feature,99997,1004,1040,0,198,Uster,ZH,Point,2696722.040,1245043.472,47.349147,8.718725
395121,Feature,99998,1004,1040,0,198,Uster,ZH,Point,2696727.666,1245050.586,47.349211,8.718801


## Match descriptions to buildingCategory &	buildingClass	

In [15]:
# Read .xslx file with the data
df_gwr_cat = pd.read_excel('gkat_translator.xlsx', sheet_name='buildingCategory')
df_gwr_class = pd.read_excel('gkat_translator.xlsx', sheet_name='buildingClass')

# Merge the DataFrames on 'buildingCategory'
df_merged = pd.merge(df, df_gwr_cat, on='buildingCategory', how='left')
df_merged = pd.merge(df_merged, df_gwr_class, on='buildingClass', how='left')

# Show data
df_merged[['egid', 'buildingCategory', 'buildingClass', 'buildingCategoryDesc', 'buildingClassDesc']].head()

,egid,buildingCategory,buildingClass,buildingCategoryDesc,buildingClassDesc
0,1,1040,1271,Gebäude mit teilweiser Wohnnutzung,Landwirtschaftliche Betriebsgebäude
1,10,1020,1110,Gebäude mit ausschliesslicher Wohnnutzung,Gebäude mit einer Wohnung
2,100,1020,1110,Gebäude mit ausschliesslicher Wohnnutzung,Gebäude mit einer Wohnung
3,1000,1030,1110,Wohngebäude mit Nebennutzung,Gebäude mit einer Wohnung
4,10000,1030,1121,Wohngebäude mit Nebennutzung,Gebäude mit zwei Wohnungen


## Plot subset of buildings

In [7]:
# Subset
df_sub = df_merged.loc[df['municipalityName'] == 'Greifensee'].sample(100).dropna()

# Create the map
m = folium.Map(location=[df_sub['latitude'].mean(), df_sub['longitude'].mean()], zoom_start=15)

# Add points to the map
for idx, row in df_sub.iterrows():
    folium.Marker(location=([row['latitude'], 
                            row['longitude']]),
                  popup=row['buildingClassDesc']).add_to(m)

# Display the map
m

### Task 5. b) Create a subset of all buildings in the municipality of Wädenswil and plot the buildings on a map.

In [8]:
# Subset
df_sub = df_merged.loc[df['municipalityName'] == 'Wädenswil'].sample(100).dropna()

# Create the map
m = folium.Map(location=[df_sub['latitude'].mean(), df_sub['longitude'].mean()], zoom_start=15)

# Add points to the map
for idx, row in df_sub.iterrows():
    folium.Marker(location=([row['latitude'], 
                            row['longitude']]),
                  popup=row['buildingClassDesc']).add_to(m)

# Display the map
m

### Task 5. c) Select and count the number of existing buildings with one apartment (GKLAS 1110) in the city of Zürich.

In [16]:
# Filter the DataFrame for buildings in Zürich with buildingClass 1110
zurich_buildings_one_apartment = df_merged[(df_merged['municipalityName'] == 'Zürich') & (df_merged['buildingClass'] == 1110)]

# Count the number of such buildings
count_zurich_buildings_one_apartment = zurich_buildings_one_apartment.shape[0]

print(f"Number of existing buildings with one apartment in the city of Zürich: {count_zurich_buildings_one_apartment}")

Number of existing buildings with one apartment in the city of Zürich: 11071


### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [8]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 6.5.0-1025-azure
Datetime: 2024-11-29 11:08:56
Python Version: 3.11.10
-----------------------------------
